In [1]:
import open3d as o3d
import numpy as np

# Example for reading point cloud

In [13]:
# I open point cloud file called Full_Point_Cloud.ply
# This is the original file from SAIE Lab
# Trying to crop it!
cloud = o3d.io.read_point_cloud("Full_Point_Cloud.ply")
# tell us about this file!
print(cloud)

PointCloud with 56569957 points.


# Example for writing point cloud

In [14]:
# I copy the contents of Full_Point_Cloud.ply to a new file called Part_1.ply
o3d.io.write_point_cloud("Part_1.ply", cloud)

True

In [4]:
# I read Part_1.ply
p1 = o3d.io.read_point_cloud("Part_1.ply")
print(p1)

PointCloud with 56569957 points.


# Notes about the point cloud
W.r.t the .TIFF file, the .PLY file is rotated about 30 degrees

# Crop Method 1: (good method so far)
part1 = p1_crop_(0,-30,0).ply
<br>part2 = p1_crop_(-200,-130,0).ply
<br>part3 = p1_crop_(300,-50,0).ply

# Mesh output: (week 3, Jan 30 ~ Feb 5, 2023)
Software: Meshlab

part1 = part1_Mesh.ply
<br>part2 = part2_Mesh.ply (normals are flipped w.r.t. view)
<br>part3 = part3_Mesh.ply (normals are flipped w.r.t. view)

Note: using 'flip normals w.r.t. view' option in normal calculation results in Bad (self-intersecting & Not abiding to point cloud) poisson mesh!

# Findings:
It's Faster to 'navigate' the mesh using Crop Method 1, because if I open the entire point cloud in Meshlab, the software slows down so much I canNot do operations (e.g. cropping) quickly because of lag

# 230209
Today I need crop out Smaller parts from p1_crop_(0,-30,0).ply
Size: 24


In [12]:
# Crop method 1
# I learn from code @ https://stackoverflow.com/questions/70449880/how-to-crop-a-point-cloud-in-open3d-by-using-a-polygon-volume
# it's by: SFA
# scale factor for the size of the box
scale = 24
# location of center
x = 20
y = -30
z = 0 # for the mesh, Z direction is 'UP'
# R is an identity matrix. 3 is argument, so dimemsion of R is (3 x 3)
R = np.identity(3)
extent = np.array([1, 1, 2])*scale
center = np.array([x, y, z])
# construct Oriented BoundingBox object
vol = o3d.geometry.OrientedBoundingBox(center,R,extent) # or you can use axis aligned bounding box class
print(vol)
# Crop point cloud 'p1', using bounding box 'vol'
p1_crop = p1.crop(vol)
o3d.io.write_point_cloud("small_crop_(" + str(x) + "," + str(y) + ","+ str(z) + ").ply", p1_crop)

OrientedBoundingBox: center: (20, -30, 0), extent: 24, 24, 48)


True

# Crop method 2: (not good, yet to figure out how bounding box location)

In [57]:
# CROP method 2 (not working atm, cropped result has 0 points, I think)
# I learn from code @ https://stackoverflow.com/questions/61269980/open3d-crop-pointcloud-with-polygon-volume
# it's by: Eric Wiener
# define volume manually: one cube = 4 vertices total
start_x = 0.0
start_y = 0.0
start_z = 0.0
cubeSize = 10000.0
corners = np.array(
        [[start_x,             start_y,             start_z],
        [start_x,             start_y + cubeSize,  start_z],
        [start_x + cubeSize,  start_y,             start_z],
        [start_x + cubeSize,  start_y + cubeSize,  start_z],
        [start_x,             start_y,            start_z + cubeSize],
        [start_x,             start_y + cubeSize,  start_z + cubeSize],
        [start_x + cubeSize,  start_y,             start_z + cubeSize],
        [start_x + cubeSize,  start_y + cubeSize,  start_z]])
print(corners)
# convert to type float64
box = corners.astype("float64")
# initialize SelectionPolygonVolume object
vol = o3d.visualization.SelectionPolygonVolume()
# I choose the "Y" axis. I made the max value the maximum Y of
# the polygon vertices and the min value the minimum Y of the
# polygon vertices.
vol.orthogonal_axis = "Y" # orientation for the box
vol.axis_max = np.max(box[:, 1])
vol.axis_min = np.min(box[:, 1])
# Set all the Y values to 0 (they aren't needed since we specified what they
# should be using just vol.axis_max and vol.axis_min).
box[:, 1] = 0
# Convert the np.array to a Vector3dVector
vol.bounding_polygon = o3d.utility.Vector3dVector(box)
# Crop w/ variable 'box'
p1_crop = vol.crop_point_cloud(p1)
o3d.io.write_point_cloud("p1_crop.ply", p1_crop)

[[    0.     0.     0.]
 [    0. 10000.     0.]
 [10000.     0.     0.]
 [10000. 10000.     0.]
 [    0.     0. 10000.]
 [    0. 10000. 10000.]
 [10000.     0. 10000.]
 [10000. 10000.     0.]]
[Open3D WARNING] Write PLY failed: point cloud has 0 points.


False

# .PLY point cloud to .PLY mesh
Software used: Meshlab
<br>Method: Poisson
<br>1. Calculate Normals
<br>&emsp;Filters -> Normals, Curvatures, and Orientation -> Compute Normals for point sets
<br>2. Turn point cloud to mesh
<br>&emsp;Filters -> Remeshing, Simplification, and Reconstruction -> Surface Reconstruction: Screened Poisson (Here, I use Default values from Feb 2023)

<br>Note: I use "&emsp;" (open markdown to see what's between the quotes!) to insert Tab (here, it inserts 4x spaces, which equals 1x tab)

# Turn .PLY Mesh into .PLY2 Mesh
We need .PLY2 file to use Shin Yoshizawa (et al.)'s Crest Line code (http://www2.riken.jp/brict/Yoshizawa/Research/Crest.html)
<br>The .PLY2 file is made of 4 parts (http://www2.riken.jp/brict/Yoshizawa/Research/PLYformat/PLYformat.html)
<br>&emsp;this file format is good for: "approximation of the smooth surfaces" (quote from website)
<br>1. Line 1: # of vertices (Integer)
<br>2. Line 2: # of triangles (Integer)
<br>3. Set of vertex coordinates
<br>&emsp;1x vertex per line
<br>&emsp;Format: (Double x, Double y, Double z) <-Remove comma!
<br>4. Set of face information
<br>&emsp;face = triangle
<br>&emsp;Format: (3, Int x, Int y, Int z) <-Remove comma!
<br>&emsp;here, x, y, and z are vertex ID, which are integers

In [2]:
mesh = o3d.io.read_triangle_mesh("part1_Mesh.ply")


In [3]:
verts = np.asarray(mesh.vertices)
verts_count = len(verts)
print("Vertices: " + str(verts_count) + " Total")
print(verts)
faces = np.asarray(mesh.triangles)
faces_count = len(faces)
print("Faces(Triangles): " + str(faces_count) + " Total")
print(faces)

Vertices: 76027 Total
[[-51.52429581 -80.11437988 -24.4055233 ]
 [-52.20049667 -79.86222076 -24.4055233 ]
 [-49.86222076 -81.09666443 -24.4055233 ]
 ...
 [ 10.38796234  -3.82233429  -0.16961479]
 [ 10.80348206  -3.82233429  -0.20088959]
 [ 11.21900177  -3.82233429  -0.24428368]]
Faces(Triangles): 151765 Total
[[    0    17     1]
 [    9    19     2]
 [    2    19     0]
 ...
 [76022 76015 71673]
 [73411 76022 71673]
 [71673 71674 73411]]


In [4]:
PLY2 = 'ply2.txt'
NEWLINE = '\n'

In [15]:
arr = np.array([[1.1,2,4],[3.1,4,7]])
print(" ".join(map(str, arr[1]))) # prints out array WITHOUT brackets


3.1 4.0 7.0


In [17]:
with open(PLY2, 'w') as f:
    f.write(str(verts_count) + NEWLINE)
    f.write(str(faces_count) + NEWLINE)
    for i in range(verts_count):
        f.write(" ".join(map(str, verts[i])) + NEWLINE)
    for j in range(faces_count):
        f.write('3 ' + " ".join(map(str, faces[i])) + NEWLINE)